# Covertype Data Set Preprocessing

In [21]:
import csv
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split

### Set the paths

In [22]:
FULL_DATASET = '../covertype.csv'
SMALL_DATASET= '../covertype_small.csv'
TRAINING_DATASET='../covertype_training.csv'
TRAINING_DATASET_WITH_MISSING = '../covertype_training_missing.csv'
EVALUATION_DATASET='../covertype_evaluation.csv'
EVALUATION_DATASET_WITH_ANOMALIES='../covertype_evaluation_anomalies.csv'
SERVING_DATASET='../covertype_serving.csv'

ORIGINAL_DATASET_PATH = 'gs://workshop-datasets/covertype/orig/covtype.data'

## Preprocess the original dataset

### Load the dataset

In [23]:
df = pd.read_csv(ORIGINAL_DATASET_PATH, header=None)
print(df.shape)
df.head()

(581012, 55)


,0,1,2,3,4,5,6,7,8,9,...,45,46,47,48,49,50,51,52,53,54
0,2596,51,3,258,0,510,221,232,148,6279,...,0,0,0,0,0,0,0,0,0,5
1,2590,56,2,212,-6,390,220,235,151,6225,...,0,0,0,0,0,0,0,0,0,5
2,2804,139,9,268,65,3180,234,238,135,6121,...,0,0,0,0,0,0,0,0,0,2
3,2785,155,18,242,118,3090,238,238,122,6211,...,0,0,0,0,0,0,0,0,0,2
4,2595,45,2,153,-1,391,220,234,150,6172,...,0,0,0,0,0,0,0,0,0,5


### Configure soil type and wilderness area domains

In [24]:
soil_type = [
"1", "C2702", "Cathedral family - Rock outcrop complex, extremely stony.",
"2", "C2703", "Vanet - Ratake families complex, very stony.",
"3", "C2704", "Haploborolis - Rock outcrop complex, rubbly.",
"4", "C2705", "Ratake family - Rock outcrop complex, rubbly.",
"5", "C2706", "Vanet family - Rock outcrop complex complex, rubbly.",
"6", "C2717", "Vanet - Wetmore families - Rock outcrop complex, stony.",
"7", "C3501", "Gothic family.",
"8", "C3502", "Supervisor - Limber families complex.",
"9", "C4201", "Troutville family, very stony.",
"10", "C4703", "Bullwark - Catamount families - Rock outcrop complex, rubbly.",
"11", "C4704", "Bullwark - Catamount families - Rock land complex, rubbly.",
"12", "C4744", "Legault family - Rock land complex, stony.",
"13", "C4758", "Catamount family - Rock land - Bullwark family complex, rubbly.",
"14", "C5101", "Pachic Argiborolis - Aquolis complex.",
"15", "C5151", "unspecified in the USFS Soil and ELU Survey.",
"16", "C6101", "Cryaquolis - Cryoborolis complex.",
"17", "C6102", "Gateview family - Cryaquolis complex.",
"18", "C6731", "Rogert family, very stony.",
"19", "C7101", "Typic Cryaquolis - Borohemists complex.",
"20", "C7102", "Typic Cryaquepts - Typic Cryaquolls complex.",
"21", "C7103", "Typic Cryaquolls - Leighcan family, till substratum complex.",
"22", "C7201", "Leighcan family, till substratum, extremely bouldery.",
"23", "C7202", "Leighcan family, till substratum - Typic Cryaquolls complex.",
"24", "C7700", "Leighcan family, extremely stony.",
"25", "C7701", "Leighcan family, warm, extremely stony.",
"26", "C7702", "Granile - Catamount families complex, very stony.",
"27", "C7709", "Leighcan family, warm - Rock outcrop complex, extremely stony.",
"28", "C7710", "Leighcan family - Rock outcrop complex, extremely stony.",
"29", "C7745", "Como - Legault families complex, extremely stony.",
"30", "C7746", "Como family - Rock land - Legault family complex, extremely stony.",
"31", "C7755", "Leighcan - Catamount families complex, extremely stony.",
"32", "C7756", "Catamount family - Rock outcrop - Leighcan family complex, extremely stony.",
"33", "C7757", "Leighcan - Catamount families - Rock outcrop complex, extremely stony.",
"34", "C7790", "Cryorthents - Rock land complex, extremely stony.",
"35", "C8703", "Cryumbrepts - Rock outcrop - Cryaquepts complex.",
"36", "C8707", "Bross family - Rock land - Cryumbrepts complex, extremely stony.",
"37", "C8708", "Rock outcrop - Cryumbrepts - Cryorthents complex, extremely stony.",
"38", "C8771", "Leighcan - Moran families - Cryaquolls complex, extremely stony.",
"39", "C8772", "Moran family - Cryorthents - Leighcan family complex, extremely stony.",
"40", "C8776", "Moran family - Cryorthents - Rock land complex, extremely stony.",
]

wilderness_area = [
"Rawah", "Rawah Wilderness Area",
"Neota", "Neota Wilderness Area",
"Commanche", "Comanche Peak Wilderness Area",
"Cache", "Cache la Poudre Wilderness Area"
]

### Map one-hot encoded values to categorical domains

In [25]:
soil = df.loc[:, 14:53].apply(lambda x: soil_type[1::3][x.to_numpy().nonzero()[0][0]], axis=1)
soil

0         C7745
1         C7745
2         C4744
3         C7746
4         C7745
          ...  
581007    C2703
581008    C2703
581009    C2703
581010    C2703
581011    C2703
Length: 581012, dtype: object

In [26]:
wilderness = df.loc[:, 10:13].apply(lambda x: wilderness_area[0::2][x.to_numpy().nonzero()[0][0]], axis=1)
wilderness

0             Rawah
1             Rawah
2             Rawah
3             Rawah
4             Rawah
            ...    
581007    Commanche
581008    Commanche
581009    Commanche
581010    Commanche
581011    Commanche
Length: 581012, dtype: object

### Create a dataset with column names and categorical values replacing one-hot encoded soil type and wilderness areas

In [27]:
COLUMN_NAMES = [
    'Elevation', 
    'Aspect', 
    'Slope', 
    'Horizontal_Distance_To_Hydrology',
    'Vertical_Distance_To_Hydrology',
    'Horizontal_Distance_To_Roadways',
    'Hillshade_9am',
    'Hillshade_Noon',
    'Hillshade_3pm',
    'Horizontal_Distance_To_Fire_Points',
    'Wilderness_Area',
    'Soil_Type',
    'Cover_Type']

df_full = pd.concat([df.loc[:, 0:9], wilderness, soil, df.loc[:, 54]], axis=1, ignore_index=True)
df_full.columns = COLUMN_NAMES
df_full

,Elevation,Aspect,Slope,Horizontal_Distance_To_Hydrology,Vertical_Distance_To_Hydrology,Horizontal_Distance_To_Roadways,Hillshade_9am,Hillshade_Noon,Hillshade_3pm,Horizontal_Distance_To_Fire_Points,Wilderness_Area,Soil_Type,Cover_Type
0,2596,51,3,258,0,510,221,232,148,6279,Rawah,C7745,5
1,2590,56,2,212,-6,390,220,235,151,6225,Rawah,C7745,5
2,2804,139,9,268,65,3180,234,238,135,6121,Rawah,C4744,2
3,2785,155,18,242,118,3090,238,238,122,6211,Rawah,C7746,2
4,2595,45,2,153,-1,391,220,234,150,6172,Rawah,C7745,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...
581007,2396,153,20,85,17,108,240,237,118,837,Commanche,C2703,3
581008,2391,152,19,67,12,95,240,237,119,845,Commanche,C2703,3
581009,2386,159,17,60,7,90,236,241,130,854,Commanche,C2703,3
581010,2384,170,15,60,5,90,230,245,143,864,Commanche,C2703,3


### Save the dataset to CSV file

In [28]:
df_full.to_csv(FULL_DATASET, header=True, index=False)

In [29]:
!head $FULL_DATASET

Elevation,Aspect,Slope,Horizontal_Distance_To_Hydrology,Vertical_Distance_To_Hydrology,Horizontal_Distance_To_Roadways,Hillshade_9am,Hillshade_Noon,Hillshade_3pm,Horizontal_Distance_To_Fire_Points,Wilderness_Area,Soil_Type,Cover_Type
2596,51,3,258,0,510,221,232,148,6279,Rawah,C7745,5
2590,56,2,212,-6,390,220,235,151,6225,Rawah,C7745,5
2804,139,9,268,65,3180,234,238,135,6121,Rawah,C4744,2
2785,155,18,242,118,3090,238,238,122,6211,Rawah,C7746,2
2595,45,2,153,-1,391,220,234,150,6172,Rawah,C7745,5
2579,132,6,300,-15,67,230,237,140,6031,Rawah,C7745,2
2606,45,7,270,5,633,222,225,138,6256,Rawah,C7745,5
2605,49,4,234,7,573,222,230,144,6228,Rawah,C7745,5
2617,45,9,240,56,666,223,221,133,6244,Rawah,C7745,5


## Create training, validation, testing and serving splits.

In [30]:
df_full = df = pd.read_csv(FULL_DATASET, dtype={'Soil_Type': object})
df_full

,Elevation,Aspect,Slope,Horizontal_Distance_To_Hydrology,Vertical_Distance_To_Hydrology,Horizontal_Distance_To_Roadways,Hillshade_9am,Hillshade_Noon,Hillshade_3pm,Horizontal_Distance_To_Fire_Points,Wilderness_Area,Soil_Type,Cover_Type
0,2596,51,3,258,0,510,221,232,148,6279,Rawah,C7745,5
1,2590,56,2,212,-6,390,220,235,151,6225,Rawah,C7745,5
2,2804,139,9,268,65,3180,234,238,135,6121,Rawah,C4744,2
3,2785,155,18,242,118,3090,238,238,122,6211,Rawah,C7746,2
4,2595,45,2,153,-1,391,220,234,150,6172,Rawah,C7745,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...
581007,2396,153,20,85,17,108,240,237,118,837,Commanche,C2703,3
581008,2391,152,19,67,12,95,240,237,119,845,Commanche,C2703,3
581009,2386,159,17,60,7,90,236,241,130,854,Commanche,C2703,3
581010,2384,170,15,60,5,90,230,245,143,864,Commanche,C2703,3


In [31]:
df_full.Soil_Type.value_counts()

C7745    115247
C7202     57752
C7756     52519
C7757     45154
C7201     33373
C4703     32634
C7746     30170
C4744     29971
C7755     25666
C7700     21278
C4758     17431
C8771     15573
C8772     13806
C4704     12410
C2705     12396
C7102      9259
C8776      8750
C2703      7525
C2717      6575
C2704      4823
C7101      4021
C6102      3422
C2702      3031
C6101      2845
C7702      2589
C6731      1899
C8703      1891
C7790      1611
C2706      1597
C4201      1147
C7709      1086
C7710       946
C7103       838
C5101       599
C7701       474
C8708       298
C3502       179
C8707       119
C3501       105
C5151         3
Name: Soil_Type, dtype: int64

In [32]:
df_5151 = df_full[df_full['Soil_Type']=='C5151']
df_no_5151 = df_full[df_full['Soil_Type']!='C5151']

In [33]:
df_5151

,Elevation,Aspect,Slope,Horizontal_Distance_To_Hydrology,Vertical_Distance_To_Hydrology,Horizontal_Distance_To_Roadways,Hillshade_9am,Hillshade_Noon,Hillshade_3pm,Horizontal_Distance_To_Fire_Points,Wilderness_Area,Soil_Type,Cover_Type
241543,2078,34,10,0,0,212,219,218,134,484,Cache,C5151,6
241544,2080,13,19,30,0,192,198,197,132,499,Cache,C5151,6
241545,2076,27,24,30,5,175,201,180,105,516,Cache,C5151,6


In [34]:
df_no_5151

,Elevation,Aspect,Slope,Horizontal_Distance_To_Hydrology,Vertical_Distance_To_Hydrology,Horizontal_Distance_To_Roadways,Hillshade_9am,Hillshade_Noon,Hillshade_3pm,Horizontal_Distance_To_Fire_Points,Wilderness_Area,Soil_Type,Cover_Type
0,2596,51,3,258,0,510,221,232,148,6279,Rawah,C7745,5
1,2590,56,2,212,-6,390,220,235,151,6225,Rawah,C7745,5
2,2804,139,9,268,65,3180,234,238,135,6121,Rawah,C4744,2
3,2785,155,18,242,118,3090,238,238,122,6211,Rawah,C7746,2
4,2595,45,2,153,-1,391,220,234,150,6172,Rawah,C7745,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...
581007,2396,153,20,85,17,108,240,237,118,837,Commanche,C2703,3
581008,2391,152,19,67,12,95,240,237,119,845,Commanche,C2703,3
581009,2386,159,17,60,7,90,236,241,130,854,Commanche,C2703,3
581010,2384,170,15,60,5,90,230,245,143,864,Commanche,C2703,3


In [36]:
df_small, df_other = train_test_split(df_no_5151, train_size=100000, stratify=df_no_5151.Cover_Type)

In [37]:
df_train, df_other = train_test_split(df_no_5151, train_size=431009, stratify=df_no_5151.Cover_Type)
df_evaluate, df_serving = train_test_split(df_other, train_size=75000, stratify=df_other.Cover_Type)
df_serving = df_serving.drop(columns=['Cover_Type'])
print(df_train.shape)
print(df_evaluate.shape)
print(df_serving.shape)

(431009, 13)
(75000, 13)
(75000, 12)


Add some missing values to the training split.

In [38]:
df_train_missing = df_train.reset_index(drop=True)
df_train_missing.loc[0:8999, 'Horizontal_Distance_To_Hydrology'] = None
df_train_missing

,Elevation,Aspect,Slope,Horizontal_Distance_To_Hydrology,Vertical_Distance_To_Hydrology,Horizontal_Distance_To_Roadways,Hillshade_9am,Hillshade_Noon,Hillshade_3pm,Horizontal_Distance_To_Fire_Points,Wilderness_Area,Soil_Type,Cover_Type
0,3191,68,8,NaN,16,3662,229,223,127,4446,Rawah,C7745,1
1,3381,351,18,NaN,49,3851,186,207,158,3461,Commanche,C8776,7
2,2537,206,6,NaN,0,1165,216,246,165,2094,Commanche,C6102,6
3,3206,135,8,NaN,8,1798,233,237,135,1556,Neota,C7202,1
4,2573,125,30,NaN,86,693,253,204,55,962,Cache,C2705,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...
431004,2525,285,9,330.0,99,618,194,240,186,323,Cache,C2717,6
431005,3139,343,10,450.0,-35,3963,200,225,163,4432,Rawah,C7202,1
431006,2877,320,12,0.0,0,5249,188,228,180,4293,Rawah,C6101,1
431007,3265,202,26,192.0,40,957,198,252,175,1033,Rawah,C7201,1


Create the evaluation split where some values of Slope are more than 90 degrees and 3 examples have 5151 code for soil type, which is not present in the training split.

In [39]:
df_evaluate_anomalies = df_evaluate.reset_index(drop=True)
df_evaluate_anomalies.loc[0:4, 'Slope'] = 110
df_evaluate_anomalies = pd.concat([df_evaluate_anomalies, df_5151])
df_evaluate_anomalies

,Elevation,Aspect,Slope,Horizontal_Distance_To_Hydrology,Vertical_Distance_To_Hydrology,Horizontal_Distance_To_Roadways,Hillshade_9am,Hillshade_Noon,Hillshade_3pm,Horizontal_Distance_To_Fire_Points,Wilderness_Area,Soil_Type,Cover_Type
0,3108,296,110,175,12,2054,200,238,178,2423,Commanche,C7756,2
1,2751,300,110,30,0,2687,202,237,175,6331,Rawah,C6101,2
2,2896,104,110,150,25,3147,238,227,119,2568,Rawah,C4744,2
3,2141,332,110,285,248,313,123,176,180,153,Cache,C4703,3
4,3059,84,110,228,24,2081,234,224,121,1406,Rawah,C7746,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
74998,3192,91,13,175,54,1572,240,218,106,2583,Rawah,C7745,1
74999,2941,90,6,255,23,3822,229,231,136,4710,Rawah,C4744,2
241543,2078,34,10,0,0,212,219,218,134,484,Cache,C5151,6
241544,2080,13,19,30,0,192,198,197,132,499,Cache,C5151,6


In [40]:
df_evaluate_anomalies.Soil_Type.value_counts()

C7745    14946
C7202     7332
C7756     6790
C7757     5641
C7201     4371
C4703     4286
C7746     3922
C4744     3898
C7755     3367
C7700     2771
C4758     2167
C8771     2007
C8772     1792
C2705     1623
C4704     1603
C7102     1214
C8776     1103
C2703      968
C2717      848
C2704      623
C7101      538
C6102      413
C2702      390
C6101      380
C7702      344
C6731      249
C8703      244
C2706      230
C7790      210
C4201      149
C7709      135
C7710      121
C7103      105
C5101       86
C8708       49
C7701       46
C3502       21
C8707       11
C3501        7
C5151        3
Name: Soil_Type, dtype: int64

### Save the splits to local files.

In [44]:
df_train.to_csv(TRAINING_DATASET, header=True, index=False)
df_small.to_csv(SMALL_DATASET, header=True, index=False)
df_train_missing.to_csv(TRAINING_DATASET_WITH_MISSING, header=True, index=False)
df_evaluate.to_csv(EVALUATION_DATASET, header=True, index=False)
df_evaluate_anomalies.to_csv(EVALUATION_DATASET_WITH_ANOMALIES, header=True, index=False)
df_serving.to_csv(SERVING_DATASET, header=True, index=False)

### Copy the splits to GCS

In [45]:
!gsutil cp $FULL_DATASET gs://workshop-datasets/covertype/full/dataset.csv
!gsutil cp $SMALL_DATASET gs://workshop-datasets/covertype/small/dataset.csv
!gsutil cp $TRAINING_DATASET gs://workshop-datasets/covertype/training/dataset.csv
!gsutil cp $TRAINING_DATASET_WITH_MISSING gs://workshop-datasets/covertype/training_missing/dataset.csv
!gsutil cp $EVALUATION_DATASET gs://workshop-datasets/covertype/evaluation/dataset.csv
!gsutil cp $EVALUATION_DATASET_WITH_ANOMALIES gs://workshop-datasets/covertype/evaluation_anomalies/dataset.csv
!gsutil cp $SERVING_DATASET gs://workshop-datasets/covertype/serving/dataset.csv

Copying file://../covertype.csv [Content-Type=text/csv]...
- [1 files][ 30.5 MiB/ 30.5 MiB]                                                
Operation completed over 1 objects/30.5 MiB.                                     
Copying file://../covertype_training.csv [Content-Type=text/csv]...
- [1 files][ 22.7 MiB/ 22.7 MiB]                                                
Operation completed over 1 objects/22.7 MiB.                                     
Copying file://../covertype_training_missing.csv [Content-Type=text/csv]...
- [1 files][ 23.4 MiB/ 23.4 MiB]                                                
Operation completed over 1 objects/23.4 MiB.                                     
Copying file://../covertype_evaluation.csv [Content-Type=text/csv]...
- [1 files][  3.9 MiB/  3.9 MiB]                                                
Operation completed over 1 objects/3.9 MiB.                                      
Copying file://../covertype_evaluation_anomalies.csv [Content-Type=text/csv